By Olufisayo Akindele Togun

#Assignment 1 (Random Forest)

* In this project you are given a dataset of housing housing price prediction. 
Dataset description is found in the given datasets.

* The goal of the project is to predict the price of a house given its attributes. 
Therefore, the problem is a regression task. 

* You need to build a random forest that consists of multiple decision trees (for regression) from the given training data set. Then, apply it on the test set and submit your code to generate predictions.
You need to build the random forest and decision trees from scratch. (I.e., it is not allowed to use existing machine learning libraries or packages such as sklearn.)

* You may use any programming language/environment of your choice, but you are required to submit the complete source code to produce the output
If you use anything other than jupyter notebook, submit an executable and run that from the main function of the jupyter notebook so that the prediction generation is automated. We can provide assistance with this.
The output (a single file with the predictions for each test instance) must be generated automatically using the approach implemented by you. Submitting predictions/code from any other source (Internet, another student, etc.) is considered cheating and will result in immediate disqualification (i.e., dismissal from the course).

##Part 1: Preprocessing and dataset analysis (20 points)

* The given dataset is quite complex, it has many attributes, and not all of them are useful! 
Training on such dataset results in a bad accuracy. And this is exactly the point! 

* "Understanding the question is half the answer". In data mining, understanding the dataset is half the answer! 

* In part 1 you need to analyze the dataset and make it clean. 
 

###Load the dataset and explore (5 points)

* Load the dataset, view the dataset and the shape of it, 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
import datetime as dt
import matplotlib.dates as mdates

from functools import reduce
import datetime as dt
from statsmodels.tsa.ar_model import AutoReg
from pandas.plotting import autocorrelation_plot
from scipy.stats import pearsonr
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder




In [2]:
price_test=pd.read_csv('sample_submission.csv')
attribute_test=pd.read_csv('housing_price_test.csv')
house_description=pd.read_csv('housing_data_description.csv', on_bad_lines='skip')
price_attribute_train=pd.read_csv('housing_price_train.csv')

In [3]:
#merge the two data set together
price_attribute_test = pd.merge(price_test,attribute_test, on= 'Id')

### Clean the dataset (10 points)

* We cannot train on a 'dirty' dataset! There are duplicated, Null, and missing values that you need to take care of!

* Drop all columns which have null values >= 70 % and drop all rows which have null values >= 70 %.

* You need to take care of categorial data!

In [4]:
def PreprocessingData(db):
    """
        TODO
        Show NULL values for each columns in dataset and thier percentage
        Drop all columns which have null values >= 70 % 
        Drop all rows which have null values >= 70 %
        Fill all missing values with the (mean, mode)

    """
    
    # checks for null values and print their percentage
    null_percent_test = db.isnull().mean()
    #print(null_percent_test)
    

    #drops all columns with null values more than 70%
    db = db.dropna(thresh=int(db.shape[0] * 0.3), axis=1)
    #db = db.dropna(thresh=int(db.shape[0] * 0.3), axis=0)
    
    
    #calculates the mean of a integers and floats in a data set
    #mean = db.select_dtypes(include=['float64','int64']).mean()

    # calculate the mode for non-numeric values
    #mode_db = db.select_dtypes(exclude=['float64','int64'])
    #if mode_db.empty:
    #    mode = None
    # else:
    #    mode = mode_db.mode().iloc[0]
    

    # combine the mean and mode dictionaries
    #mean.update(mode)
    
    # fill missing values with the mean or mode of each column depending on column type
    #db = db.fillna(mean)
    miss_val= db.isnull().sum().sort_values(ascending=False)

    miss_val = pd.DataFrame(data=db.isnull().sum().sort_values(ascending=False), columns=['MissingValueCount'])

    columns_with_missing_values = miss_val[miss_val['MissingValueCount'] > 0].index

    price_train_filtered = db[columns_with_missing_values]

    columns_with_missing_values = miss_val[miss_val['MissingValueCount'] > 0].index
    
    for col in columns_with_missing_values:
        mode = db[col].mode().values[0]
        db[col].fillna(mode, inplace=True)



    #How to find the numeric columns in pandas
    numerics = ['int16','int32','int64','float32','float64']
    numerics_cols= list(db.select_dtypes(include = numerics))

    #convert all numeric columns to intergers
    for col in numerics_cols: 
        db[col] = pd.to_numeric(db[col], errors='coerce')

    
    return(db)


In [5]:
price_test=PreprocessingData(price_attribute_test)
price_train=PreprocessingData(price_attribute_train)


C:\Users\fisay\AppData\Local\Temp\ipykernel_6752\177736019.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db[col].fillna(mode, inplace=True)
C:\Users\fisay\AppData\Local\Temp\ipykernel_6752\177736019.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db[col].fillna(mode, inplace=True)
C:\Users\fisay\AppData\Local\Temp\ipykernel_6752\177736019.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db[col].fillna(mode, inplace=True

In [6]:
#verification to show that all data types have no null values 
miss_val= price_train.isnull().sum().sort_values(ascending=False)
miss_val = pd.DataFrame(data=price_train.isnull().sum().sort_values(ascending=False), columns=['MissingValueCount'])
miss_val[miss_val['MissingValueCount']>0]

,MissingValueCount


In [7]:
print(price_train.dtypes)

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 77, dtype: object


### Correlations! (5 points)

* Now we have a clean dataset, but not all attributes are useful! 

* Display the corrlation between all features and the sales price. This will show you which feature affects sales price more. You may use *corr()* function. 

* Choose the most correlated features, and remove others. 

In [8]:
def cor_select(db):
        # input a primary key
    primary_key = 'SalePrice'

    # input column to be exempt
    exempt_col = 'Id'

   # calculate the correlation matrix
    corr_matrix = db.corr().abs()

    # sort the correlation matrix by the correlation with the primary key column
    sorted_corr = corr_matrix.sort_values(by=primary_key, axis=0, ascending=False)
    

    # select the top 10 columns with the highest correlation
    top_correlated = sorted_corr.drop(primary_key).head(25)
    

    # store the columns names 
    top_columns = top_correlated.index
    print(top_columns)

    # add the exempt and primarycolumn to the list
    top_columns = top_columns.append(pd.Index([exempt_col]))
    top_columns = top_columns.append(pd.Index([primary_key]))
    
    # drop the other columns
    db = db[top_columns]

    return(db)

In [9]:
price_attribute_train[price_attribute_train.columns].corr().abs()['SalePrice'][:].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
KitchenAbvGr     0.135907
EnclosedPorch    0.128578
ScreenPorch      0.111447
PoolArea         0.092404
MSSubClass       0.084284
OverallCond      0.077856
MoSold           0.046432
3SsnPorch        0.044584
YrSold           0.028923
LowQualFinSF     0.025606
Id               0.021917
MiscVal          0.021190
BsmtHalfBath     0.016844
BsmtFinSF2       0.011378
Name: SalePr

In [10]:
price_train=cor_select(price_train)
price_test= cor_select(price_test)

Index(['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd',
       'MasVnrArea', 'Fireplaces', 'GarageYrBlt', 'BsmtFinSF1', 'LotFrontage',
       'WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'HalfBath', 'LotArea',
       'BsmtFullBath', 'BsmtUnfSF', 'BedroomAbvGr', 'KitchenAbvGr',
       'EnclosedPorch'],
      dtype='object')
Index(['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       '1stFlrSF', 'YearBuilt', 'FullBath', 'YearRemodAdd', 'MasVnrArea',
       'TotRmsAbvGrd', 'GarageYrBlt', 'BsmtFinSF1', 'Fireplaces',
       'LotFrontage', 'OpenPorchSF', 'BsmtFullBath', 'LotArea', 'HalfBath',
       'WoodDeckSF', '2ndFlrSF', 'OverallCond', 'BsmtUnfSF', 'BedroomAbvGr',
       'EnclosedPorch'],
      dtype='object')


In [11]:
print(price_train.dtypes)
#it is observed that non of the data is an obj

OverallQual        int64
GrLivArea          int64
GarageCars         int64
GarageArea         int64
TotalBsmtSF        int64
1stFlrSF           int64
FullBath           int64
TotRmsAbvGrd       int64
YearBuilt          int64
YearRemodAdd       int64
MasVnrArea       float64
Fireplaces         int64
GarageYrBlt      float64
BsmtFinSF1         int64
LotFrontage      float64
WoodDeckSF         int64
2ndFlrSF           int64
OpenPorchSF        int64
HalfBath           int64
LotArea            int64
BsmtFullBath       int64
BsmtUnfSF          int64
BedroomAbvGr       int64
KitchenAbvGr       int64
EnclosedPorch      int64
Id                 int64
SalePrice          int64
dtype: object


In [12]:
price_train

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,...,OpenPorchSF,HalfBath,LotArea,BsmtFullBath,BsmtUnfSF,BedroomAbvGr,KitchenAbvGr,EnclosedPorch,Id,SalePrice
0,7,1710,2,548,856,856,2,8,2003,2003,...,61,1,8450,1,150,3,1,0,1,208500
1,6,1262,2,460,1262,1262,2,6,1976,1976,...,0,0,9600,0,284,3,1,0,2,181500
2,7,1786,2,608,920,920,2,6,2001,2002,...,42,1,11250,1,434,3,1,0,3,223500
3,7,1717,3,642,756,961,1,7,1915,1970,...,35,0,9550,1,540,3,1,272,4,140000
4,8,2198,3,836,1145,1145,2,9,2000,2000,...,84,1,14260,1,490,4,1,0,5,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,1647,2,460,953,953,2,7,1999,2000,...,40,1,7917,0,953,3,1,0,1456,175000
1456,6,2073,2,500,1542,2073,2,7,1978,1988,...,0,0,13175,1,589,3,1,0,1457,210000
1457,7,2340,1,252,1152,1188,2,9,1941,2006,...,60,0,9042,0,877,4,1,0,1458,266500
1458,5,1078,1,240,1078,1078,1,5,1950,1996,...,0,0,9717,1,0,2,1,112,1459,142125


In [13]:
price_test

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,YearBuilt,FullBath,YearRemodAdd,MasVnrArea,...,LotArea,HalfBath,WoodDeckSF,2ndFlrSF,OverallCond,BsmtUnfSF,BedroomAbvGr,EnclosedPorch,Id,SalePrice
0,5,896,1.0,730.0,882.0,896,1961,1,1961,0.0,...,11622,0,140,0,6,270.0,2,0,1461,115000
1,6,1329,1.0,312.0,1329.0,1329,1958,1,1958,108.0,...,14267,1,393,0,6,406.0,3,0,1462,158000
2,5,1629,2.0,482.0,928.0,928,1997,2,1998,0.0,...,13830,1,212,701,5,137.0,3,0,1463,173000
3,6,1604,2.0,470.0,926.0,926,1998,2,1998,20.0,...,9978,1,360,678,6,324.0,3,0,1464,178000
4,8,1280,2.0,506.0,1280.0,1280,1992,2,1992,0.0,...,5005,0,0,0,5,1017.0,2,0,1465,220000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,4,1092,0.0,0.0,546.0,546,1970,1,1970,0.0,...,1936,1,0,546,7,546.0,3,0,2915,92000
1455,4,1092,1.0,286.0,546.0,546,1970,1,1970,0.0,...,1894,1,0,546,5,294.0,3,0,2916,88000
1456,5,1224,2.0,576.0,1224.0,1224,1960,1,1996,0.0,...,20000,0,474,0,7,0.0,4,0,2917,137500
1457,5,970,0.0,0.0,912.0,970,1992,1,1992,0.0,...,10441,0,80,0,5,575.0,3,0,2918,146500


It is observed that non of the columns selected using the correlation are neither categorical or norminal.
The code below is stored for future reference

In [14]:
#le = LabelEncoder()
#y = le.fit_transform(y)

#onehotencoder = OneHotEncoder()
#X = onehotencoder.fit_transform(X).toarray()

In [15]:


#def labelencoder(db):
#    le = LabelEncoder()
#    for col in db.columns:
#        if price_train[col].dtype == 'object':
#            price_train[col] = le.fit_transform(db[col].astype(str))


## Part 2: Decision Tree (45 points)
#### Building a Decision Tree:
A Decision tree consists of nodes connected by edges. A decision tree is typically, a binary tree, which has the following properties:
- One node is marked as Root node
- Every node other than the root has a parent node
- Each node can have at most 2 child nodes (left edge & right edge)
- Leaf node is the node which contains pure data or when we reach to the maximum depth 

To create the decision tree model for scratch you need to create two classes (a class for the node, for example "class DecisionNode():" and a class for Decision Tree model, for example "class RegressionDecisionTree():")


1- DecisionNode class used to save some values for each node we do the spliting on it until we reach the leaf node
so we will save the following values for the node:
- feature: feature index.
- threshold: the value we used to split the data on.
- value: the average value for the leaf node.
- True_Branch: if the condition is true.
- False_Branch: if the condition is false.

In [16]:
class DecisionNode():
    def __init__(self, feature_idx=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature_idx = feature_idx # index of the feature that is used
        self.threshold = threshold     # threshold value for feature when making the decision
        self.value = value # Average value if the node is a leaf in the tree
        self.true_branch = true_branch # the node we go to if decision returns True
        self.false_branch = false_branch # the node we go to if decision returns False

# Decision Tree Class
This Class consists the following functions:
<ol>
<li> <b>build_tree</b>: used to create the decision tree nodes</li> 
<li> <b>calc_variance_reduction</b> : measure the impurity by using variance reduction measure (like MSE) </li> 
the function takes three parameters (parentRec: the records for the target before split,and the left and right records after splitting. This function used to measure the impurity for each node and decide if we will split or not.
<li> <b>majority_vote</b>: used to calculate values for the leaf nodes records which equal to the mean of these records.</li> 
<li><b>split_by_feature</b>: this function take the feature and the threshold and check if the feature is numerical so it split the records into two node (true which is the left edge and false which is the right edge)
if the feature is categorical so it split where the values equal to the threshold</li>
<li> <b>fit</b>: Used to train the dataset after spliting the data into two part x: features, y: target</li>
<li><b>predict_value</b>: used to predict the value for each record, it is a recursive method to find the leaf node that corresponds to prediction
<li><b>predict</b>: take all records for the test data and iterate into each record to predit the y(target) value and save the result into a prediction list. 

In [17]:
class RegressionDecisionTree():
    # constructor
    def __init__(self, min_VarianceReduction=1e-7, max_depth=5):        
        self.root = None # root of this tree
        self.min_VarianceReduction = min_VarianceReduction # minimum VarianceReduction to allow splitting
        # used to stopping conductions
        self.max_depth = max_depth # maximum depth the tree grows to
 

    # used to create the decision tree nodes
    def build_tree(self, X, y, current_depth=0):
        # we will use decision dictionary to save the feature and the threshold we build the tree on 
        decision = None
        # we will use subtrees dictionary to save the feature and the threshold we build the tree on 
        subtrees = None
        largest_variance_Reduction = 0
        # add y as last column of X
        df = pd.concat((X, y), axis=1)
        n_rows, n_features = X.shape
        if current_depth <= self.max_depth:
            # iterate through every feature
            for feature_idx in range(n_features):
                # values of that column
                feature_values = X.iloc[:, feature_idx]                
                unique_values = feature_values.unique()                
                for threshold in unique_values:
                    X_trueEdge, X_falseEdge = self.split_by_feature(df, feature_idx, threshold)
                    if len(X_trueEdge) > 0 and len(X_falseEdge) > 0:
                        y_true = X_trueEdge.iloc[:,-1]
                        y_false = X_falseEdge.iloc[:,-1]                        
                        # Calculate impurity
                        VarianceRed = self.Calc_variance_reduction(y, y_true, y_false)
                        # Keep track of which feature gave the largest information gain
                        if VarianceRed > largest_variance_Reduction:
                            largest_variance_Reduction = VarianceRed
                            decision = {"feature_idx":feature_idx, "threshold":threshold}
                            subtrees = {"X_true":X_trueEdge.iloc[:,:-1],
                                        "y_true":y_true,
                                        "X_false":X_falseEdge.iloc[:,:-1],
                                        "y_false":y_false}

        # we will construct new branch of tree if the variance_Reduction is larger than minimum variance_Reduction that we've defined
        if largest_variance_Reduction > self.min_VarianceReduction:
            true_branch = self.build_tree(subtrees["X_true"], subtrees["y_true"], current_depth+1)
            false_branch = self.build_tree(subtrees["X_false"], subtrees["y_false"], current_depth+1)
            return DecisionNode(feature_idx=decision["feature_idx"], threshold=decision["threshold"], true_branch=true_branch, false_branch=false_branch)

        # at leaf node we calculate the mean for the records
        leaf_value = self.majority_vote(y)
        return DecisionNode(value=leaf_value)
                        
    # measure the impurity by using variance reduction measure (like MSE)
    # left_edgeRec= True edge: where condition is true
    # Right_edgeRec= False edge: where condition is false
    def Calc_variance_reduction(self, parentRec, left_edgeRec, Right_edgeRec):   
        # return the VarReduction = variance for parent - (Weight * var(leftEdge) + Weight * var(RightEdge)    
        weight_left = len(left_edgeRec)/len(parentRec)
        weight_right = len(Right_edgeRec)/len(parentRec)
        Var_red = np.var(parentRec) - ((weight_left*np.var(left_edgeRec))+ (weight_right*np.var(Right_edgeRec)))
        return (Var_red)
        
    
    def majority_vote(self, Y): 
        # return the majority_vote for the leaf nodes 
        vote= np.mean(Y)
        return(vote)
        
             
    def split_by_feature(self, db, feature_idx, threshold):
    
        # split the data into left_edge & right_edge depends one specified feature and the threshold
        # return left & right edges
        #if isinstance(threshold, int) or isinstance(threshold, float):
        if isinstance(threshold, int) or isinstance(threshold, float):
            X_true = db[db.iloc[:,feature_idx] >= threshold]
            X_false = db[db.iloc[:,feature_idx] < threshold]
        # if the feature is categorical
        else:
            X_true = db[db.iloc[:,feature_idx] == threshold]
            X_false = db[~db.iloc[:,feature_idx] == threshold]
        return X_true, X_false
        

    
    # Used to train the dataset after spliting the data into x: features, y: target
    def fit(self, X, y):
        self.root = self.build_tree(X, y)


    def predict_value(self, xTest, tree=None):
        # recursive method to find the leaf node that corresponds to prediction
        
        if tree is None:
            tree = self.root
        if tree.value is not None:
            return tree.value
        feature_value = xTest[tree.feature_idx]
        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.true_branch
        elif feature_value == tree.threshold:
            branch = tree.true_branch
        return self.predict_value(xTest, branch)

    # to predict the value we need to pass the all records for features and we save the prediction for each records into a list
    def predict(self, XTest):
        y_pred = []
        for idx, row in XTest.iterrows():           
            y_pred.append(self.predict_value(row.values))
        return y_pred


- To Check the Accuracy for our prediction we use CalcAccuracy function which take the actual values for the test dataset and the predicted values and apply the RMSE formula.

In [18]:
def CalcAccuracy(Actual_Y, Predicted_y):
    
    actual_values=Actual_Y.to_numpy()
    predicted_values=np.array(Predicted_y)
    error= abs(actual_values-predicted_values)
    mse=np.mean(np.square(error))
    rel_err=mse/np.mean(np.square(actual_values))
    accuracy=1-rel_err
    return(accuracy)

In [19]:
test_tree=RegressionDecisionTree(min_VarianceReduction=1e-7, max_depth=5)
test_tree.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])
prdicted=test_tree.predict(price_train.iloc[:,:-1])
CalcAccuracy((price_train.iloc[:,-1]),prdicted)
#train data

0.940292578774546

In [20]:
test_tree=RegressionDecisionTree(min_VarianceReduction=1e-6, max_depth=5)
test_tree.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])
prdicted=test_tree.predict(price_test.iloc[:,:-1])
CalcAccuracy((price_test.iloc[:,-1]),prdicted)
#test data

0.8159622753785021

- Build decision tree model
- Fit the model
- Predict the values from test data 

## Part 3: Random Forest (20 points)
#### Random forest class
- the Class consist of the following functions:
<ul>
    <li>Constructor: consists of the subset data (Training & Testing) dataset after preprocessing and a list of deciceion tree objects </li>    
    <li>Subsampling: Bagging we will take random sample with replacement for the Training dataset </li>
    <li>build_model: first make subsample for the training dataset, then split the data into featurespart(X) and targetpart(Y), then take 10 samples of the feature part, finally build the decision tree (fit), this function take the number of DT that we want to build</li>
    <li>predict: take the test dataset and make the prediction for the target field in all the tree in the random forest then take the mean for the prediction in each tree, finally add the mean of prediction to a list of predition </li>

</ul>

In [21]:
class RandomForest:
    def __init__(self, number_of_trees=10, sample_size_ratio=0.5, max_depth=2,min_VarianceReduction=1e-7):
        self.Traindata = None  # training data set (loaded into memory)
        self.Testdata = None  # Test data set for prediction        
        self.trees = []  # list of decision trees 
        self.number_of_trees=number_of_trees
        self.sample_size_ratio=sample_size_ratio
        self.max_depth=max_depth
        self.min_VarianceReduction=min_VarianceReduction
     # This function generate a subsample with replacement
    def __subsampling(self, train_set):       
        sample_number = round(len(train_set) * self.sample_size_ratio)
        return  train_set.sample(n = sample_number, replace=True)
        
        
        
    def build_model(self, train_set):
        for i in range(self.number_of_trees):
            TrainingSample = self.__subsampling(train_set)
            tree=RegressionDecisionTree(min_VarianceReduction=self.min_VarianceReduction,max_depth= self.max_depth)
            tree.fit(TrainingSample.iloc[:,:-1], TrainingSample.iloc[:,-1])
            self.trees.append(tree)

            
               
    def predict(self, test_set):
        predictions=np.array([tree.predict(test_set) for tree in self.trees])
        return np.mean(predictions, axis=0) #column mean for the decision tree
        # Predict for each item in the list 
        # Calculate the mean 
    

### Create Random Forest

* Create 10 Decision Tree in the randomforest
* Train the random forest with the dataset
* Use the created random forest to predict the test dataset 

In [22]:
RF=RandomForest(number_of_trees=10, min_VarianceReduction=1e-7,sample_size_ratio=0.4, max_depth=2)
RF.build_model(price_train)
rf_prd=RF.predict(price_train.iloc[:,:-1])
CalcAccuracy((price_train.iloc[:,-1]),rf_prd)
#train data

0.9158212462762418

In [23]:
RF=RandomForest(number_of_trees=10, min_VarianceReduction=1e-7,sample_size_ratio=0.4, max_depth=2)
RF.build_model(price_train)
rf_prd=RF.predict(price_test.iloc[:,:-1])
CalcAccuracy((price_test.iloc[:,-1]),rf_prd)
#test data

0.8160254164603982

##Part4: Comparison! (15 points)

Now that you have finished implementing your Random Forest, it's time for some experiments and analysis! 

* Use the Random Forest in the scikit-learn library and train it on the same dataset. 

* Compare the accuracy given by your Random Forest to the scikit-learn one. 

* Increase the number of trees in your Random Forest. Does it improve the accuracy? 

* Make a table for comparing your Random Forest accuracy with different number of trees with the scikit-learn one. What is your conclusion? 

In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [25]:
sk_tree=DecisionTreeRegressor(max_depth=5,min_impurity_decrease=0.0000001)

In [26]:

sk_tree.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])
sk_predicted=sk_tree.predict(price_train.iloc[:,:-1])
CalcAccuracy((price_train.iloc[:,-1]),sk_predicted)
#train

0.9771595883339913

In [27]:

sk_tree.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])#fit with train data
sk_prd=sk_tree.predict(price_test.iloc[:,:-1])
CalcAccuracy((price_test.iloc[:,-1]),sk_prd)
#test

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


0.9454337924589592

In [28]:
sk_RF = RandomForestRegressor(n_estimators=10,max_depth=2)

In [29]:
sk_RF.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])
sk_predicted=sk_RF.predict(price_train.iloc[:,:-1])
CalcAccuracy((price_train.iloc[:,-1]),sk_predicted)
#train

0.9512123051588064

In [30]:
sk_RF.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])#fit with train data
sk_predicted=sk_RF.predict(price_test.iloc[:,:-1])
CalcAccuracy((price_test.iloc[:,-1]),sk_predicted)
#test

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


0.9366992009555213

In [31]:
no_tree=[1,2,4,6,8,10,20,30,40,50,60,70,80,90,100]
values=np.array([])
for i in no_tree:
    sk_RF = RandomForestRegressor(n_estimators=i,max_depth=2)
    RF=RandomForest(number_of_trees=i, min_VarianceReduction=1e-7,sample_size_ratio=0.4, max_depth=2)
    print(i)
    sk_RF.fit(price_train.iloc[:,:-1], price_train.iloc[:,-1])#fit with train data
    sk_prd=sk_RF.predict(price_test.iloc[:,:-1])
    cal1=CalcAccuracy((price_test.iloc[:,-1]),sk_prd)


    RF.build_model(price_train)
    rf_prd=RF.predict(price_test.iloc[:,:-1])
    cal2=CalcAccuracy((price_test.iloc[:,-1]),rf_prd)
    
    values=np.append(values,np.array([i,cal1,cal2]))




c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


1
2


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


4


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


...

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


90


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


100


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- OverallCond
Feature names seen at fit time, yet now missing:
- KitchenAbvGr

  warnings.warn(message, FutureWarning)


In [32]:
values=np.reshape(values, (15, 3))
df= pd.DataFrame(values, columns=['No of trees','Sklearn','Random forest model'])


In [33]:
df

,No of trees,Sklearn,Random forest model
0,1.0,0.923638,0.819382
1,2.0,0.934670,0.826787
2,4.0,0.940952,0.828439
3,6.0,0.937920,0.816712
4,8.0,0.933771,0.814743
5,10.0,0.936366,0.829402
6,20.0,0.936035,0.823291
7,30.0,0.938173,0.825723
8,40.0,0.938031,0.824050
9,50.0,0.938073,0.824956


It can be seen that accuracy increase between 2 to 4 trees for sklearn and the model and begins to drop